In [1]:
from keras.layers import Dense, Conv1D, Conv2D, LSTM, GRU, Flatten, Dropout, Input, Reshape, BatchNormalization
from keras.layers import MaxPool1D, MaxPool2D, Embedding, Bidirectional, TimeDistributed, concatenate
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adadelta, Adagrad, Adam, Adamax, Nadam, RMSprop, SGD
import keras.backend as K
import numpy as np
import os
import re
import pandas as pd
from random import choice
import seaborn as sns
from sklearn.utils.extmath import softmax
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook

Using TensorFlow backend.


In [2]:
optimizer_pool = {
        Adadelta: {'learning_rate': [1.0, 0.99, 0.95, 0.9, 0.85, 0.8]}, 
        Adagrad: {'learning_rate': [0.1, 0.05, 0.01, 0.005, 0.001]},
        Adam: {'learning_rate': [0.01, 0.005, 0.001, 0.0005, 0.0001]},
        Adamax: {'learning_rate': [0.02, 0.005, 0.002, 0.0005, 0.0002]},
        Nadam: {'learning_rate': [0.02, 0.005, 0.002, 0.0005, 0.0002]},
        RMSprop: {'learning_rate': [0.01, 0.005, 0.001, 0.0005, 0.0001]},
        SGD: {'learning_rate': [0.1, 0.05, 0.01, 0.005, 0.001], 
              'nesterov': [True], 'momentum': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]}
}

LAYERS = {"Dense": Dense, "Conv1D": Conv1D, "Conv2D": Conv2D, 
          "LSTM": LSTM, "GRU": GRU, "Flatten": Flatten, 
          "Dropout": Dropout, "Input": Input, "Reshape": Reshape, 
          "BatchNormalization": BatchNormalization, "MaxPooling1D": MaxPool1D, 
          "MaxPooling2D": MaxPool2D, "Embedding": Embedding, 
          "Bidirectional": Bidirectional, "TimeDistributed": TimeDistributed}

parameter_pool = {
    "initializers": [
            "Zeros",
            "Ones",
            "RandomNormal",
            "RandomUniform",
            "TruncatedNormal",
            "VarianceScaling",
            "Orthogonal",
            "lecun_uniform",
            "glorot_normal",
            "glorot_uniform",
            "he_normal",
            "lecun_normal",
            "he_uniform"
        ],
    "activations": [
            "elu",
            "relu",
            "softmax",
            "selu",
            "softplus",
            "softsign",
            "tanh",
            "sigmoid",
            "hard_sigmoid",
            "exponential",
            "linear"
        ],
    "regularizers": [
            "l1",
            "l2",
            "l1_l2"
        ],
    
    "constraints": [
            "MaxNorm",
            "NonNeg",
            "UnitNorm",
            "MinMaxNorm"
        ]
    
}

layer_parameters = {
    "Dense": {
                "units": [2, 4, 6, 8, 10, 12,14, 16],
                "activation": parameter_pool["activations"], 
                "use_bias": [False, True], 
                "kernel_initializer": parameter_pool["initializers"], 
                "bias_initializer": parameter_pool["initializers"], 
                "kernel_regularizer": parameter_pool["regularizers"], 
                "bias_regularizer": parameter_pool["regularizers"], 
                "activity_regularizer": parameter_pool["regularizers"], 
                "kernel_constraint": parameter_pool["constraints"], 
                "bias_constraint": parameter_pool["constraints"]
            },
    "Conv1D": {
                "filters": [2, 4, 6, 8, 10, 12, 14, 16], 
                "kernel_size": [(1,), (2,), (3,), (4,), (5,), (6,), (7,), (8,)], 
                "padding": ["same"], 
                # "data_format": [None, "channels_last", "channels_first"], 
                "activation": parameter_pool["activations"], 
                "use_bias": [True, False], 
                "kernel_initializer": parameter_pool["initializers"], 
                "bias_initializer": parameter_pool["initializers"], 
                "kernel_regularizer": parameter_pool["regularizers"], 
                "bias_regularizer": parameter_pool["regularizers"], 
                "activity_regularizer": parameter_pool["regularizers"], 
                "kernel_constraint": parameter_pool["constraints"], 
                "bias_constraint": parameter_pool["constraints"]
            },
    "Conv2D": {
                "filters": [2, 4, 6, 8, 10, 12, 14, 16], 
                "kernel_size": [(1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6), (7, 7), (8, 8)], 
                # "padding": ["valid", "causal", "same"], 
                # "data_format": [None, "channels_last", "channels_first"], 
                "activation": parameter_pool["activations"], 
                "use_bias": [True, False], 
                "kernel_initializer": parameter_pool["initializers"], 
                "bias_initializer": parameter_pool["initializers"], 
                "kernel_regularizer": parameter_pool["regularizers"], 
                "bias_regularizer": parameter_pool["regularizers"], 
                "activity_regularizer": parameter_pool["regularizers"], 
                "kernel_constraint": parameter_pool["constraints"], 
                "bias_constraint": parameter_pool["constraints"]
            },
    "GRU": {
                "units": [2, 4, 6, 8, 10, 12, 14, 16], 
                "activation": parameter_pool["activations"], 
                "recurrent_activation": parameter_pool["activations"], 
                "use_bias": [True, False], 
                "kernel_initializer": parameter_pool["initializers"], 
                "recurrent_initializer": parameter_pool["initializers"], 
                "bias_initializer": parameter_pool["initializers"], 
                "kernel_regularizer": parameter_pool["regularizers"], 
                "recurrent_regularizer": parameter_pool["regularizers"], 
                "bias_regularizer": parameter_pool["regularizers"], 
                "activity_regularizer": parameter_pool["regularizers"], 
                "kernel_constraint": parameter_pool["constraints"], 
                "recurrent_constraint": parameter_pool["constraints"], 
                "bias_constraint": parameter_pool["constraints"], 
                "dropout": [0.0, 0.1, 0.2, 0.3, 0.4, 0.5], 
                "recurrent_dropout": [0.0, 0.1, 0.2, 0.3, 0.4, 0.5], 
                "implementation": [1, 2], 
                "return_sequences": [True], 
                # "return_state": [True, False], 
                "go_backwards": [True, False], 
                # "stateful": [True, False], 
                "unroll": [True, False], 
                "reset_after": [True, False]
            },
    "LSTM": {
                "units": [2, 4, 6, 8, 10, 12, 14, 16], 
                "activation": parameter_pool["activations"], 
                "recurrent_activation": parameter_pool["activations"], 
                "use_bias": [True, False], 
                "kernel_initializer": parameter_pool["initializers"], 
                "recurrent_initializer": parameter_pool["initializers"], 
                "bias_initializer": parameter_pool["initializers"], 
                "unit_forget_bias": [True, False],
                "kernel_regularizer": parameter_pool["regularizers"], 
                "recurrent_regularizer": parameter_pool["regularizers"], 
                "bias_regularizer": parameter_pool["regularizers"], 
                "activity_regularizer": parameter_pool["regularizers"], 
                "kernel_constraint": parameter_pool["constraints"], 
                "recurrent_constraint": parameter_pool["constraints"], 
                "bias_constraint": parameter_pool["constraints"], 
                "dropout": [0.0, 0.1, 0.2, 0.3, 0.4, 0.5], 
                "recurrent_dropout": [0.0, 0.1, 0.2, 0.3, 0.4, 0.5], 
                "implementation": [1, 2], 
                "return_sequences": [True], 
                # "return_state": [True, False], 
                "go_backwards": [True, False], 
                # "stateful": [True, False], 
                "unroll": [True, False], 
            },
    "Dropout": {
                "rate": [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]
            },
    "Flatten": {
                "data_format": [None, "channels_last", "channels_first"]
            },
    "MaxPooling1D": {
                "pool_size": [2, 3, 4, 5, 6, 7, 8], 
                # "padding": ["valid", "causal", "same"],
                # "data_format": [None, "channels_last", "channels_first"]
            }, 
    "MaxPooling2D": {
                "pool_size": [(2, 2), (3, 3), (4, 4), (5, 5), (6, 6), (7, 7), (8, 8)], 
                # "padding": ["valid", "causal", "same"],
                # "data_format": [None, "channels_last", "channels_first"]
            } 
}

def get_random_layer_parameters(layer):
            layer_name = re.sub(r'<.+\'(.+)\'>', r'\1', str(layer)).split(".")[-1]
            random_parameters = {key: choice(value) for key, value in layer_parameters[layer_name].items()}
            return layer(**random_parameters)

In [46]:
class RandNet(object):
    
    def __init__(self, feature_data_shape, label_data_shape,
                 num_classes=1,
                 get_random_layer_parameters=get_random_layer_parameters,
                 optimizer_pool=optimizer_pool,
                 layer_types=LAYERS, init_hl=True):
        
        """
        Creates a randomly initialized neural net in keras with 
        functional API Model and is used for initial state for
        a modified NEAT-algorithm
        
        parameters:
        - feature_data_shape: tuple, shape of the feature data
        - label_data_shape: tuple, shape of the label data 
                            -> NO one-hot-encoding needed
        => easiest way: feature_data / label_data as numpy array 
           -> use shape attribute
        - num_classes: int, number of classes in label data
           -> defaults to: 1 -> binary classification
        - get_random_layer_parameters: function, randomly initializes layer
        - optimizer_pool: dict, parameters for keras optimizers
        """
        
        # data configs
        self.feature_data_shape = feature_data_shape
        self.label_data_shape = label_data_shape
        self.feature_shape_length = len(self.feature_data_shape)
        self.input_shape = tuple(list(self.feature_data_shape)[1:])
        self.output_shape = tuple(list(self.label_data_shape)[1:])
        self.num_classes = num_classes
        self.layer_names = [str(lt) for lt in layer_types.values()]
        self.layer_types = list(layer_types.values())
        self.core_layers = [Dense, Conv1D, Conv2D, LSTM, GRU]
        self.optimizer_pool = optimizer_pool
        self.get_random_layer_parameters = get_random_layer_parameters
        self.optimizers = ["sgd", "rmsprop", "adadelta", "adam", 
                           "adamax", "nadam"]
               
        # init net
        self.input_layer = Input(shape=self.input_shape)
        if init_hl:
            self.hidden_layers = pd.DataFrame(data={"parallel_1": \
                                                    [get_random_layer_parameters(choice(self.core_layers))]})
        else:
            self.hidden_layers = pd.DataFrame(data={"parallel_1": \
                                                    []})
        if num_classes is None or 1 <= num_classes <= 2:
            self.output_layer = Dense(1)
        else:
            self.output_layer = Dense(num_classes)
        self.output_tensors = []
        self.insertion_count = 0
        self.deletion_count = 0
        self.mutation_count = 0
        self.input_tensor = self.input_layer
        self.output_tensor = None
        
    def initialize_random_layer(self, layers=[Dense, Conv1D, Conv2D, LSTM, GRU]):
        return self.get_random_layer_parameters(choice(layers))
        
    def insertion(self, layer, axis=0):
        
        cols = self.hidden_layers.columns
        df_length = len(self.hidden_layers)
        if axis == 0:
            # vertical insertion
            rand_col = choice(cols)
            if isinstance(layer, tuple(self.core_layers)):
                self.hidden_layers.loc[df_length, rand_col] = layer
            else:
                self.hidden_layers.loc[df_length, rand_col] = self.get_random_layer_parameters(layer)
            
        elif axis == 1:
            # horizontal insertion
            parallel_count = re.search(r'_(\d+)', cols[-1])
            if parallel_count:
                parallel_count = int(parallel_count.group(1))
            else:
                raise TypeError("No column number found in column string.")
            
            if isinstance(layer, tuple(self.core_layers)):
                self.hidden_layers["parallel_{}".format(parallel_count + 1)] = \
                [layer] + ([np.nan] * (df_length - 1))
            else:
                self.hidden_layers["parallel_{}".format(parallel_count + 1)] = \
                [self.get_random_layer_parameters(layer)] + ([np.nan] * (df_length - 1))            
            
        else:
            raise ValueError("Axis-Error: please check the value of the axis parameter.")
            
    def modify(self, layer, input_tensor):
        # insert some modification layers like MaxPooling etc.
        rand_num = np.random.uniform(0, 1)
        layer_name = str(type(layer))
        
        # some maxpooling
        if layer_name == str(Conv1D):
            output_tensor = self.get_random_layer_parameters(MaxPool1D)(input_tensor)
        elif layer_name == str(Conv2D) and len(input_tensor.shape) == 3:
            output_tensor = self.get_random_layer_parameters(MaxPool1D)(input_tensor)
        elif layer_name == str(Conv2D) and len(input_tensor.shape) > 3:
            output_tensor = self.get_random_layer_parameters(MaxPool2D)(input_tensor)
        else:
            output_tensor = input_tensor
            
        # some Dropout layer
        if rand_num <= 0.5:
            output_tensor = self.get_random_layer_parameters(Dropout)(output_tensor)
        # some BatchNormalization layer
        elif 0.5 < rand_num <= 0.75:
            output_tensor = BatchNormalization()(output_tensor)
            
        return output_tensor
    
    def bidirectional(self, recurrent_layer):
        # make a bidirectional connection to a recurrent layer
        return Bidirectional(recurrent_layer)
    
    def timedistribute(self, dense_layer):
        # make a timedistributed connection to a dense layer
        return TimeDistributed(dense_layer)
    
    def send_tensor_and_reshape(self, modify=True):
        # walk through the hidden_layers dataframe and 
        # connect the layers with the input tensor and
        # insert reshape layer if necessary
        output_tensors = []
        for parallel in self.hidden_layers.columns:
            tensor = None
            for i, layer in enumerate(self.hidden_layers[parallel].values.tolist()):
                if str(layer) == 'nan':
                    continue
                if i == 0:
                    tensor = self.inject_reshape(layer, self.input_layer)
                else:
                    tensor = self.inject_reshape(layer, tensor)
                # add some modification layers
                if modify:
                    if np.random.uniform(0, 1) <= 0.5:
                        tensor = self.modify(layer, tensor)
            if tensor is not None:
                output_tensors.append(tensor)
        self.output_tensors = output_tensors
        
    def concat_output_tensors(self):
        # concatenate the output tensors to one 
        # final tensor for last dense layer
        reshaped_tensors = []
        for ot in self.output_tensors:
            try:
                reshaped_tensors.append(Reshape((-1,))(ot))
            except:
                continue
        if len(reshaped_tensors) >= 2:
            return concatenate(reshaped_tensors)
        elif len(reshaped_tensors) == 1:
            return reshaped_tensors[0]
        else:
            return reshaped_tensors
                    
    def inject_reshape(self, layer, input_tensor):
        layer_type = str(type(layer))
        tensor_shape = input_tensor.shape
        tensor_shape_length = len(tensor_shape)
        if layer_type == str(Dense):
            return layer(input_tensor)
        elif layer_type in [str(LSTM), str(GRU), str(Conv1D)]:
            # (batch_size, features, time_steps)
            if tensor_shape_length == 3:
                return layer(input_tensor)
            elif tensor_shape_length == 2:
                reshaped = Reshape((tensor_shape[1], 1))(input_tensor)
                return layer(reshaped)
            elif tensor_shape_length > 3:
                reshaped = Reshape((tensor_shape[1], -1))
                return layer(reshaped)
            else:
                raise ValueError("The Input tensor must have more than 2 dimensions for recurrent layers."\
                                 "Given:  {}.".format(tensor_shape))
        elif layer_type == str(Conv2D):
            if tensor_shape_length == 2:
                new_layer = self.get_random_layer_parameters(Conv1D)
                reshaped = Reshape((tensor_shape[1], 1))(input_tensor)
                print("2 dim to Conv1D: " + str(reshaped.shape))
                return new_layer(reshaped)
            elif tensor_shape_length == 3:
                reshaped = Reshape((tensor_shape[1], tensor_shape[2], 1))(input_tensor)
                print("3 dim to 4 dim: " + str(reshaped.shape))
                return layer(reshaped)
            elif tensor_shape_length >= 4:
                reshaped = Reshape((tensor_shape[1], tensor_shape[2], -1))(input_tensor)
                return layer(reshaped)
            else:
                raise ValueError("The Input tensor must have more than 3 dimensions for Conv2D layers."\
                                 "Given:  {}.".format(tensor_shape))
                    
    
    def create_model(self, print_summary=True):
        # randomly pick optimizer
        opti = choice(list(self.optimizer_pool.keys()))
        opti = opti(**{key: choice(value) for key, value in self.optimizer_pool[opti].items()})
        
        # connect net parts
        self.send_tensor_and_reshape()
        self.output_tensor = self.output_layer(self.concat_output_tensors())
        
        # initialize model
        model = Model(inputs=self.input_tensor, outputs=self.output_tensor)
        if print_summary:
            model.summary()
        if self.num_classes > 2:
            model.compile(optimizer=opti, 
                          loss="sparse_categorical_crossentropy", 
                          metrics=["sparse_categorical_accuracy"])
        elif self.num_classes in [1, 2]:
            model.compile(optimizer=opti, 
                          loss="binary_crossentropy", 
                          metrics=["binary_accuracy"])
        else:
            raise ValueError("Some Error in choosing the loss function ...")

        return model

In [4]:
bc_data = load_breast_cancer()
bc_features = bc_data['data']
bc_labels = bc_data['target']
X_train, X_test, y_train, y_test = train_test_split(bc_features, bc_labels, test_size=0.2, random_state=42)

In [5]:
randnet = RandNet(feature_data_shape=X_train.shape, label_data_shape=y_train.shape)

In [6]:
randnet.insertion(layer=choice(randnet.core_layers), axis=0)

In [7]:
randnet.insertion(layer=choice(randnet.core_layers), axis=1)

In [8]:
randnet.hidden_layers

,parallel_1,parallel_2
0,<keras.layers.convolutional.Conv2D object at 0...,<keras.layers.convolutional.Conv1D object at 0...
1,<keras.layers.recurrent.LSTM object at 0x7f0d3...,NaN


In [9]:
model = randnet.create_model()

2 dim to Conv1D: (None, 30, 1)
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 30)           0                                            
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 30, 1)        0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 30, 6)        12          reshape_1[0][0]                  
__________________________________________________________________________________________________
reshape_2 (Reshape)             (None, 30, 1)        0           input_1[0][0]                    
_____________________________________________________________

In [10]:
model.fit(X_train, y_train, batch_size=int(round(X_train.shape[0] * 0.05)), epochs=10, validation_data=(X_test, y_test))

Train on 455 samples, validate on 114 samples
Epoch 1/10
455/455 [==============================] - 2s 4ms/step - loss: 3247.7070 - binary_accuracy: 0.3890 - val_loss: 25.1402 - val_binary_accuracy: 0.8684
Epoch 2/10
455/455 [==============================] - 0s 359us/step - loss: 16804.3765 - binary_accuracy: 0.4659 - val_loss: 26.5064 - val_binary_accuracy: 0.6228
Epoch 3/10
455/455 [==============================] - 0s 369us/step - loss: 15325.9287 - binary_accuracy: 0.4681 - val_loss: 32.5796 - val_binary_accuracy: 0.6228
Epoch 4/10
455/455 [==============================] - 0s 364us/step - loss: 5013.3030 - binary_accuracy: 0.4681 - val_loss: 37.6340 - val_binary_accuracy: 0.0351
Epoch 5/10
455/455 [==============================] - 0s 365us/step - loss: 185360.0526 - binary_accuracy: 0.4022 - val_loss: 31.5003 - val_binary_accuracy: 0.6228
Epoch 6/10
455/455 [==============================] - 0s 354us/step - loss: 820.7546 - binary_accuracy: 0.4593 - val_loss: 24.0272 - val_binar

In [11]:
del randnet
del model

In [53]:
class EvoNet(object):
    
    def __init__(self, feature_data, label_data, start_population=10, 
                 num_classes=1, threshold=0.99, max_generations=10, layer_dict=LAYERS):
        
        self.breeding_pool = {}
        self.feature_data = np.array(feature_data)
        self.label_data = np.array(label_data)
        self.start_population = start_population
        self.num_classes = num_classes
        self.data_amount = self.feature_data.shape[0]
        self.gen_count = 1
        self.offspring_count = self.start_population - 2
        self.max_generations = max_generations
        self.threshold = threshold
        self.layer_dict = layer_dict
        self.core_layers = [Dense, Conv1D, Conv2D, LSTM, GRU]
        # batch_size depending on amount of data
        self.batch_size = round(0.05 * self.data_amount)
            
        # train and test data split
        if self.data_amount < 1000:
            self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.feature_data, self.label_data, test_size=0.33, random_state=42)
        else:
            self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.feature_data, self.label_data, test_size=0.2, random_state=42)
        
    def generate_initial_generation(self):
        key = "gen_1"
        first_population = []
        for i in range(self.start_population):
            # generate randnets for modelling
            randnet = RandNet(self.feature_data.shape, self.label_data.shape, num_classes=self.num_classes)
            first_population.append({"model": randnet.create_model(), "checkpoint": None, "train_history": None})
                    
        self.breeding_pool[key] = first_population
    
    def mutate(self):
        pass
    
    def evaluate(self):
        """
        Train each model over 10 epochs and choose best 
        accuracy for ranking.
        Appends top 2 tuple at gen value (population) in dict
        """
        directory = ".model_results/"
        if not os.path.exists(directory):
            os.makedirs(directory)
        for gen, population in self.breeding_pool.items():
            if population[0]["checkpoint"] is None:
                for individuum in population:
                    checkpoint = ModelCheckpoint(directory + "{}_{}.h5".format(gen, individuum["model"].name), 
                                                 verbose=False, monitor=individuum["model"]._compile_metrics[0],
                                                 save_best_only=True, mode='max')
                    individuum["train_history"] = individuum["model"].fit(self.X_train, self.y_train, epochs=10, 
                                                                          validation_data=(self.X_test, self.y_test), 
                                                                          callbacks=[checkpoint], verbose=False)
                    individuum["checkpoint"] = checkpoint
                best_epochs = sorted([individuum["checkpoint"].best for individuum in population], reverse=True)
                top_2 = tuple(sorted([individuum for individuum in population if individuum["checkpoint"].best in best_epochs[:2]], key=lambda x: x["checkpoint"].best, reverse=True))
                self.breeding_pool[gen].append(top_2)
    
    def mate(self):
        previous_key = "gen_{}".format(self.gen_count)
        previous_gen = self.breeding_pool[previous_key]
        new_key = "gen_{}".format(self.gen_count + 1)
        top_2 = self.breeding_pool[previous_key][-1]
        male, female = (top_2[0]["model"], top_2[1]["model"])
        male_hidden_layers = self.get_hidden_layers(male)
        female_hidden_layers = self.get_hidden_layers(female)
        
        if self.gen_count == 1:
            
            rand_num_1 = np.random.uniform(0, 1)
            # offspring
            offsprings = []
            for j in tqdm_notebook(range(self.offspring_count)):
                for z in range(5):
                    try:
                        randnet = RandNet(self.feature_data.shape, self.label_data.shape, 
                                          num_classes=self.num_classes, init_hl=True)

                        if rand_num_1 <= 0.5:
                            rand_num_2 = np.random.uniform(0, 1)
                            # insert male hidden layers
                            self.insert_hidden_layers(randnet, male_hidden_layers)
                            # insert female hidden layers
                            if rand_num_2 <= 0.5:
                                self.insert_hidden_layers(randnet, female_hidden_layers)
                            else:
                                self.insert_hidden_layers(randnet, female_hidden_layers, axis=1)
                            offsprings.append({"model": randnet.create_model(), "checkpoint": None, "train_history": None})
                        else:
                            rand_num_2 = np.random.uniform(0, 1)
                            # insert male hidden layers
                            self.insert_hidden_layers(randnet, female_hidden_layers)
                            # insert female hidden layers
                            if rand_num_2 <= 0.5:
                                self.insert_hidden_layers(randnet, male_hidden_layers)
                            else:
                                self.insert_hidden_layers(randnet, male_hidden_layers, axis=1)
                            offsprings.append({"model": randnet.create_model(), "checkpoint": None, "train_history": None})
                        break
                    except ValueError:
                        continue
                # adding offspring to new generation in breeding_pool 
                self.breeding_pool[new_key] = offsprings
        else:
            # define some layer mating and crossing over
            pass
        
        self.gen_count += 1
        
    def insert_hidden_layers(self, randnet, hidden_layers, axis=0, modify=False):
        for hl in hidden_layers:
            randnet.insertion(hl, axis)
            
    # randnet as parameter instead of model    
    def get_hidden_layers(self, model):
        model_layers = model.get_config()["layers"]
        model_hidden_layers = model_layers[1:-1]
        recovered_layers = []
        for hidden_layer in model_hidden_layers:
            new_layer = self.layer_dict[hidden_layer["class_name"]].from_config(hidden_layer["config"])
            if str(type(new_layer)) in [str(cl) for cl in self.core_layers]:
                recovered_layers.append(new_layer)
            
        return recovered_layers
        
    def layer_class_to_class_name(layer_class):
        return re.sub(r'<.+\'(.+)\'>', r'\1', str(layer_class))
    
    def select(self):
        gen_key = "gen_{}".format(self.gen_count)
        actual_gen = self.breeding_pool[gen_key]
        initial_gen_length = len(self.breeding_pool[gen_key])
        for c in range(initial_gen_length):
            if c == initial_gen_length - 1:
                break
            else:
                del self.breeding_pool[gen_key][0]

In [54]:
evonet = EvoNet(bc_features, bc_labels, start_population=5)

In [55]:
evonet.generate_initial_generation()

Model: "model_34"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_40 (InputLayer)        (None, 30)                0         
_________________________________________________________________
reshape_69 (Reshape)         (None, 30, 1)             0         
_________________________________________________________________
conv1d_19 (Conv1D)           (None, 30, 10)            40        
_________________________________________________________________
max_pooling1d_11 (MaxPooling (None, 5, 10)             0         
_________________________________________________________________
reshape_70 (Reshape)         (None, 50)                0         
_________________________________________________________________
dense_48 (Dense)             (None, 1)                 51        
Total params: 91
Trainable params: 91
Non-trainable params: 0
______________________________________________________________

In [56]:
evonet.evaluate()

In [57]:
evonet.breeding_pool["gen_1"][-1][0]["checkpoint"].best

0.8346457

In [58]:
evonet.mate()

Model: "model_39"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_45 (InputLayer)           (None, 30)           0                                            
__________________________________________________________________________________________________
reshape_79 (Reshape)            (None, 30, 1)        0           input_45[0][0]                   
__________________________________________________________________________________________________
lstm_8 (LSTM)                   (None, 30, 4)        80          reshape_79[0][0]                 
__________________________________________________________________________________________________
reshape_80 (Reshape)            (None, 30, 1)        0           input_45[0][0]                   
___________________________________________________________________________________________

In [59]:
evonet.breeding_pool

{'gen_1': [{'model': <keras.engine.training.Model at 0x7f0c55422950>,
   'checkpoint': <keras.callbacks.callbacks.ModelCheckpoint at 0x7f0c54ef18d0>,
   'train_history': <keras.callbacks.callbacks.History at 0x7f0c5485ae10>},
  {'model': <keras.engine.training.Model at 0x7f0c553c3290>,
   'checkpoint': <keras.callbacks.callbacks.ModelCheckpoint at 0x7f0c548b7510>,
   'train_history': <keras.callbacks.callbacks.History at 0x7f0c541e7f90>},
  {'model': <keras.engine.training.Model at 0x7f0c552b5290>,
   'checkpoint': <keras.callbacks.callbacks.ModelCheckpoint at 0x7f0c5427b250>,
   'train_history': <keras.callbacks.callbacks.History at 0x7f0c534e6bd0>},
  {'model': <keras.engine.training.Model at 0x7f0c550d9a10>,
   'checkpoint': <keras.callbacks.callbacks.ModelCheckpoint at 0x7f0c538a8cd0>,
   'train_history': <keras.callbacks.callbacks.History at 0x7f0c538963d0>},
  {'model': <keras.engine.training.Model at 0x7f0c54fcfa50>,
   'checkpoint': <keras.callbacks.callbacks.ModelCheckpoint at

In [ ]:
evonet.select()

In [19]:
del evonet